In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=True)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), eval_set[0][1].shape[0]
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(Astro=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'OPC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
        target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/terminator/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/terminator/tmp/MBP/model_epoch_9.pth', 'MBP')
important_genes

{'Astro': {},
 'Endo': {},
 'OPC': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.015687732957303524,
  20: 0.0,
  21: 0.0,
  22: -0.010061983950436115,
  23: 0.0,
  24: -0.0016694604637450539,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: -0.003754078410565853,
  34: -0.029492103029042482,
  35: 0.01671419618651271,
  36: 0.0,
  37: -0.003247547196224332,
  38: -0.017961908131837845,
  39: 0.006601895707717631,
  40: 0.0,
  41: 0.006352096912451088,
  42: 0.0,
  43: 0.0015002626460045576,
  44: 0.0,
  45: -0.013185465708374977,
  46: -0.030522165820002556,
  47: -0.04089357051998377,
  48: 0.0,
  49: 0.0,
  50: -0.0007292748859981657,
  51: 0.0,
  52: 0.0,
  53: 0.01092919334769249,
  54: -0.008913300931453705,
  55: -0.036558542400598526,
  56: 0.0,
  57: -0.0075962417759001255,


In [4]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/terminator/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'OPC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_MBP(OPC)_terminator.csv')
genes

,value,name
index,,
11485,49.650328,CSMD1
15794,47.600561,MALAT1
18428,36.781998,PCDH9
14031,28.369490,PCDH15
10961,25.284598,LHFPL3
...,...,...
12070,-5.718944,RALYL
5320,-7.005462,CADM2
3302,-7.816939,CTNNA2
